Authorization

1.   List item
2.   List item



In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Libraries

In [0]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Variables

In [0]:
weeks_dict = {1:['2019-01-27','2019-01-28','2019-01-29','2019-01-30','2019-01-31','2019-02-01','2019-02-02'],
              2:['2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-07','2019-02-08','2019-02-09'],
              3:['2019-02-10','2019-02-11','2019-02-12','2019-02-13','2019-02-14','2019-02-15','2019-02-16'],
              4:['2019-02-17','2019-02-18','2019-02-19','2019-02-20','2019-02-21','2019-02-22','2019-02-23'],
              5:['2019-02-24','2019-02-25','2019-02-26','2019-02-27','2019-02-28','2019-03-01','2019-03-02'],
              6:['2019-03-03','2019-03-04','2019-03-05','2019-03-06','2019-03-07','2019-03-08','2019-03-09'],
              7:['2019-03-10','2019-03-11','2019-03-12','2019-03-13','2019-03-14','2019-03-15','2019-03-16'],
              8:['2019-03-17','2019-03-18','2019-03-19','2019-03-20','2019-03-21','2019-03-22','2019-03-23'],
              9:['2019-03-24','2019-03-25','2019-03-26','2019-03-27','2019-03-28','2019-03-29','2019-03-30'],
              10:['2019-03-31','2019-04-01','2019-04-02','2019-04-03','2019-04-04','2019-04-05','2019-04-06'],
              11:['2019-04-07','2019-04-08','2019-04-09','2019-04-10','2019-04-11','2019-04-12','2019-04-13'],
              12:['2019-04-14','2019-04-15','2019-04-16','2019-04-17','2019-04-18','2019-04-19','2019-04-20'],
              13:['2019-04-21','2019-04-22','2019-04-23','2019-04-24','2019-04-25','2019-04-26','2019-04-27'],
              14:['2019-04-28','2019-04-29','2019-04-30','2019-05-01','2019-05-02','2019-05-03','2019-05-04'],
              15:['2019-05-05','2019-05-06','2019-05-07','2019-05-08','2019-05-09','2019-05-10','2019-05-11'],
              16:['2019-05-12','2019-05-13','2019-05-14','2019-05-15','2019-05-16','2019-05-17','2019-05-18']}

In [0]:
student_namelist = ['s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12',
                 's13','s14','s15','s16','s17','s18','s19','s20','s21','s22','s23','s24']
instructor_namelist = ['i1','i2','i3','i4','i5']
namelist =  student_namelist + instructor_namelist

In [0]:
directory_processed = './drive/Shared drives/2018-Makerspace-Personalization/Analyses/CS205-Parallel Computing/Data/Processed Data'


Helper Functions

In [0]:
# import files from directory
def import_files(directory_in):
  
  files = []

  for file in os.listdir(directory_in):
      if file.endswith('.csv'):
          files.append(file)

  files.sort()

  return(files)

In [0]:
# select features & names and combine all data into single dataframe

def concat_df(features,names,directory_in,files):
  
  df_raw = pd.DataFrame()

  for file in files:
      file_input = os.path.join(directory_in,file)
      df_temp = pd.read_csv(file_input,index_col=0)
      df_raw = pd.concat( [df_raw, df_temp], ignore_index=True)
      print(df_raw.shape)
  
  df_raw = df_raw[features]
  df_raw = df_raw.loc[df_raw['person_name'].isin(names)]
  df_raw.dropna(inplace=True)
  df_raw.reset_index(inplace=True,drop=True)
  
  # round timestamp
  df_raw['minute'] = pd.to_datetime(df_raw['timestamp']).dt.round('min')
  
  return(df_raw)

In [0]:
# function to calculate euclidean distance 
def map_distance(x1,y1,x2,y2):
    dist = np.sqrt((x1-x2)**2+(y1-y2)**2)
    return dist

In [0]:
# create df_pos_function to generate proximity feature

def df_pos_function(df_raw,dist,namelist,instructor_namelist,student_namelist):
    
    minute_index = list(set(df_raw['minute']))
    minute_index.sort()

    df_pos = df_raw.groupby(['minute','person_name'])['Head_x','Head_y'].mean()
    df_pos['identity'] = ""
    df_pos['interaction'] = ""
    df_pos['type'] = ""
    df_pos = pd.concat([df_pos, pd.DataFrame(columns = namelist)],axis=1)
    df_pos.fillna(0,inplace=True)

    for i in range(len(minute_index)):
        time = minute_index[i]
        df_temp = df_pos.xs(time)
        df_temp = df_temp.reset_index()

    # Interaction
        for j in range(df_temp.shape[0]-1):
            c = df_temp.shape[0]-j-1
            for k in range(c):
                d = k + j + 1
                name1 = df_temp.iloc[j][0]
                name2 = df_temp.iloc[d][0]
                name1_x = df_temp.iloc[j][1]
                name1_y = df_temp.iloc[j][2]
                name2_x = df_temp.iloc[d][1]
                name2_y = df_temp.iloc[d][2]

                # Test for interaction
                if map_distance(name1_x,name1_y,name2_x,name2_y) <= dist:
                    df_pos.loc[(time,name1),name2] = 1
                    df_pos.loc[(time,name2),name1] = 1
                    
                    if df_pos.loc[(time,name1),'interaction'] == "":
                        df_pos.loc[(time,name1),'interaction'] = name2
                    else:
                        df_pos.loc[(time,name1),'interaction'] = df_pos.loc[(time,name1),'interaction'] + ' ' + name2
                  
                    if df_pos.loc[(time,name2),'interaction'] == "":
                        df_pos.loc[(time,name2),'interaction'] = name1
                    else:
                        df_pos.loc[(time,name2),'interaction'] = df_pos.loc[(time,name2),'interaction'] + ' ' + name1
                
    df_pos = df_pos.replace(0,np.nan)
    df_pos = df_pos.dropna(axis=1,how="all")
    df_pos = df_pos.replace(np.nan,0)
    df_pos.reset_index(level='minute',inplace=True)
    df_pos.reset_index(inplace=True)

    # Test for type of interaction
    for i in range(df_pos.shape[0]):
        if df_pos.at[i,'interaction'] == "":
            continue
        elif df_pos.at[i,'interaction'] in instructor_namelist :
            df_pos.at[i,'type'] = 'Instructor'
        elif df_pos.at[i,'interaction'] in student_namelist :
            df_pos.at[i,'type'] = 'Student'
        else:
            b = df_pos.at[i,'interaction'].split(' ')
            instructor_indicator = 0
            student_indicator = 0
            total_indicator = 0
            
            for j in range(len(b)):
                if b[j] in instructor_namelist:
                    instructor_indicator = 2
                elif b[j] in student_namelist:
                    student_indicator = 1
                else:
                    df_pos.at[i,'type'] = 'Not in List'
            
            total_indicator = instructor_indicator + student_indicator 
            
            if total_indicator == 0:
                df_pos.at[i,'type']= '0?'
            elif total_indicator == 1:
                df_pos.at[i,'type'] = 'Students'
            elif total_indicator == 2:
                df_pos.at[i,'type'] = 'Instructors'
            elif total_indicator == 3:
                df_pos.at[i,'type'] = 'Mixed'
            else:
                df_pos.at[i,'type'] = 'Add?'
    
    # Fill Identity
    for i in range(df_pos.shape[0]):
        if df_pos.at[i,'person_name'] in instructor_namelist :
            df_pos.at[i,'identity'] = 'Instructor'
        elif df_pos.at[i,'person_name'] in student_namelist :
            df_pos.at[i,'identity'] = 'Student'
        else:
            df_pos.at[i,'identity'] = 'Not in List'
            
    return df_pos

In [0]:
# function to add week column
def add_week(df_original,weeks_dict):
  df = df_original.copy()
  #df['date'] = pd.to_datetime(df['timestamp']).dt.date 
  for index, row in df.iterrows():
    for key,value in weeks_dict.items():
      if str(df.at[index,'date']) in value:
        df.at[index,'week'] = key
        break
  return df

df_raw

In [0]:
directory_in = './drive/Shared drives/2018-Makerspace-Personalization/Analyses/CS205-Parallel Computing/Data/Cleaned Data'
files = import_files(directory_in)
files.remove('df_names.csv')
print(files)


['2019-02-14.csv', '2019-02-15.csv', '2019-02-16.csv', '2019-02-18.csv', '2019-02-19.csv', '2019-02-20.csv', '2019-02-21.csv', '2019-02-22.csv', '2019-02-23.csv', '2019-02-25.csv', '2019-02-26.csv', '2019-02-27.csv', '2019-02-28.csv', '2019-03-01.csv', '2019-03-02.csv', '2019-03-03.csv', '2019-03-04.csv', '2019-03-05.csv', '2019-03-06.csv', '2019-03-07.csv', '2019-03-08.csv', '2019-03-10.csv', '2019-03-11.csv', '2019-03-12.csv', '2019-03-13.csv', '2019-03-14.csv', '2019-03-15.csv', '2019-03-16.csv', '2019-03-19.csv', '2019-03-20.csv', '2019-03-21.csv', '2019-03-22.csv', '2019-03-23.csv', '2019-03-25.csv', '2019-03-26.csv', '2019-03-27.csv', '2019-03-28.csv', '2019-03-29.csv', '2019-03-30.csv', '2019-03-31.csv', '2019-04-01.csv', '2019-04-02.csv', '2019-04-03.csv', '2019-04-04.csv', '2019-04-05.csv', '2019-04-06.csv', '2019-04-07.csv', '2019-04-08.csv', '2019-04-09.csv', '2019-04-10.csv', '2019-04-11.csv', '2019-04-12.csv', '2019-04-14.csv', '2019-04-15.csv', '2019-04-16.csv', '2019-04-

In [0]:
features = ['person_name','timestamp', 'Head_x','Head_y']
df_raw = concat_df(features,namelist,directory_in,files)

(22602, 80)
(41517, 80)
(41518, 80)
(63160, 80)
(83750, 80)
(117196, 80)
(133131, 80)
(147014, 80)
(147048, 80)
(163690, 80)
(182169, 80)
(182191, 80)
(192267, 80)
(195614, 80)
(195649, 80)
(196593, 80)
(203886, 80)
(204284, 80)
(211701, 80)
(224584, 80)
(233831, 80)
(249629, 80)
(265242, 80)
(266716, 80)
(267123, 80)
(287244, 80)
(306237, 80)
(306343, 80)
(313811, 80)
(316927, 80)
(324197, 80)
(327879, 80)
(327882, 80)
(338262, 80)
(352893, 80)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (79) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(402110, 80)
(426841, 80)
(444494, 80)
(444511, 80)
(452548, 80)
(473518, 80)
(491810, 80)
(506468, 80)
(512017, 80)
(516338, 80)
(516420, 80)
(518741, 80)
(536732, 80)
(553841, 80)
(571191, 80)
(589085, 80)
(594951, 80)
(598620, 80)
(623629, 80)
(643984, 80)
(644056, 80)
(666209, 80)
(676891, 80)
(686602, 80)
(694100, 80)
(718980, 80)
(735819, 80)
(770522, 80)
(790823, 80)
(805204, 80)
(805652, 80)
(818294, 80)
(856162, 80)
(891512, 80)
(931081, 80)
(961673, 80)
(976696, 80)
(976770, 80)
(977119, 80)
(978839, 80)
(980151, 80)
(980166, 80)


In [0]:
df_raw

,person_name,timestamp,Head_x,Head_y,minute
0,i3,2019-02-14 06:09:02,0.136456,0.147472,2019-02-14 06:09:00
1,i3,2019-02-14 06:09:04,0.002692,0.067774,2019-02-14 06:09:00
2,i3,2019-02-14 06:09:12,0.608696,0.710797,2019-02-14 06:09:00
3,i3,2019-02-14 06:09:14,0.625466,0.712613,2019-02-14 06:09:00
4,i3,2019-02-14 06:09:16,1.255889,-0.180068,2019-02-14 06:09:00
...,...,...,...,...,...
980161,s19,2019-05-08 14:41:07,1.501168,0.225024,2019-05-08 14:41:00
980162,s17,2019-05-08 14:51:26,0.902343,-0.309815,2019-05-08 14:51:00
980163,s17,2019-05-08 14:51:27,0.913455,-0.330128,2019-05-08 14:51:00
980164,s19,2019-05-08 14:56:01,1.132544,0.122221,2019-05-08 14:56:00


In [0]:
# save df_raw
file_output = os.path.join(directory_processed,'df_raw.csv')
df_raw.to_csv(file_output)

df_pos

In [0]:
# output df_pos as csv
dist = 1
df_pos = df_pos_function(df_raw,dist,namelist,instructor_namelist,student_namelist)
df_pos['type'].replace('', 'Individual',inplace=True)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


In [0]:
# Check for errors
print(set(df_pos['identity']))
print(set(df_pos['type']))

{'Instructor', 'Student'}
{'Mixed', 'Instructor', 'Individual', 'Students', 'Instructors', 'Student'}


In [0]:
df_pos

,person_name,minute,Head_x,Head_y,identity,interaction,type,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,i1,i2,i3,i4,i5
0,i3,2019-02-14 06:09:00,0.700724,0.466483,Instructor,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,i3,2019-02-14 06:10:00,0.807450,0.582403,Instructor,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,i3,2019-02-14 06:11:00,0.806823,0.582437,Instructor,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,i3,2019-02-14 06:12:00,0.809035,0.584945,Instructor,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,i3,2019-02-14 06:13:00,0.810987,0.585139,Instructor,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67739,s17,2019-05-08 14:06:00,2.623022,-0.556934,Student,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67740,s19,2019-05-08 14:41:00,1.501168,0.225024,Student,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67741,s17,2019-05-08 14:51:00,0.907899,-0.319971,Student,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67742,s19,2019-05-08 14:56:00,1.132544,0.122221,Student,,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# save df_pos
file_output = os.path.join(directory_processed,'df_pos.csv')
df_pos.to_csv(file_output)

df_daily

In [0]:
# import df_pos from directory
file_input = os.path.join(directory_processed,'df_pos.csv')
df_pos = pd.read_csv(file_input,index_col=0)

df_pos.head()

,person_name,minute,Head_x,Head_y,identity,interaction,type,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,i1,i2,i3,i4,i5
0,i3,2019-02-14 06:09:00,0.700724,0.466483,Instructor,NaN,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,i3,2019-02-14 06:10:00,0.807450,0.582403,Instructor,NaN,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,i3,2019-02-14 06:11:00,0.806823,0.582437,Instructor,NaN,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,i3,2019-02-14 06:12:00,0.809035,0.584945,Instructor,NaN,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,i3,2019-02-14 06:13:00,0.810987,0.585139,Instructor,NaN,Individual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# output df_daily as csv

df_daily = pd.DataFrame()
df_daily['name'] = df_pos['person_name']
df_daily['identity'] = df_pos['identity']
df_daily['minute'] = pd.to_datetime(df_pos['minute'])
df_daily['date'] = df_daily['minute'].dt.date
df_daily['time'] = 1
df_daily['type'] = df_pos['type']
df_daily = df_daily.groupby(['name','identity','date','type'])['time'].sum()
df_daily = df_daily.reset_index()


In [0]:
# Check for errors
print(len(set(df_daily['name'])))
print(set(df_daily['type']))
print(set(df_daily['identity']))


29
{'Mixed', 'Instructor', 'Individual', 'Students', 'Instructors', 'Student'}
{'Instructor', 'Student'}


In [0]:
df_daily

,name,identity,date,type,time
0,i1,Instructor,2019-02-14,Individual,2
1,i1,Instructor,2019-02-14,Instructor,3
2,i1,Instructor,2019-02-14,Mixed,2
3,i1,Instructor,2019-02-14,Student,8
4,i1,Instructor,2019-02-14,Students,2
...,...,...,...,...,...
5545,s9,Student,2019-05-06,Student,29
5546,s9,Student,2019-05-06,Students,5
5547,s9,Student,2019-05-07,Individual,7
5548,s9,Student,2019-05-07,Mixed,5


In [0]:
# save df_daily
file_output = os.path.join(directory_processed,'df_daily.csv')
df_daily.to_csv(file_output)

df_weekly

In [0]:
# import df_daily from directory
file_input = os.path.join(directory_processed,'df_daily.csv')
df_daily = pd.read_csv(file_input,index_col=0)


In [0]:
# add column for week
df_weekly = add_week(df_daily,weeks_dict)

In [0]:
# output df_weekly as csv
df_weekly.drop(['date'], axis=1, inplace=True)
df_weekly = df_weekly.groupby(['name','identity','week','type'])['time'].sum()
df_weekly = df_weekly.reset_index()

In [0]:
# Check for errors
print(len(set(df_weekly['name'])))
print(set(df_weekly['type']))
print(set(df_weekly['identity']))

29
{'Mixed', 'Instructor', 'Individual', 'Students', 'Instructors', 'Student'}
{'Instructor', 'Student'}


In [0]:
df_weekly

,name,identity,week,type,time
0,i1,Instructor,3.0,Individual,3
1,i1,Instructor,3.0,Instructor,4
2,i1,Instructor,3.0,Mixed,3
3,i1,Instructor,3.0,Student,9
4,i1,Instructor,3.0,Students,4
...,...,...,...,...,...
1825,s9,Student,14.0,Students,47
1826,s9,Student,15.0,Individual,37
1827,s9,Student,15.0,Mixed,5
1828,s9,Student,15.0,Student,29


In [0]:
# save df_weekly
file_output = os.path.join(directory_processed,'df_weekly.csv')
df_weekly.to_csv(file_output)